In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values
import json

pd.set_option("display.max_columns", 100)

In [2]:
xqrs = pd.read_csv('../to_csv/xqrs.csv').drop('Unnamed: 0', axis=1)
xqrs.head()

,Set,Categorie Montage,Dossier Patient,Patient,Session,File N°,interval_index,interval_start_time,mean_nni,sdnn,...,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1,label
0,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,1.0,10000.0,537.109375,56.359305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,2.0,20000.0,602.213542,51.251884,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,3.0,30000.0,564.453125,33.125314,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,train,01_tcp_ar,86,8643,s001_2012_02_22,t000,4.0,40000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [3]:
xqrs['Date'] = xqrs.Session.str[-10:]
xqrs['Date'] = xqrs.Date.str.replace('_', '/')
xqrs['Date'] =  pd.to_datetime(xqrs['Date'], format='%Y/%m/%d').dt.strftime('%d/%m/%Y')

xqrs.Session = xqrs.Session.str[:4]

cols = xqrs.columns.tolist()
cols = cols[:5] + ['Date'] + cols[5:-1]

xqrs = xqrs[cols]

xqrs.head()

,Set,Categorie Montage,Dossier Patient,Patient,Session,Date,File N°,interval_index,interval_start_time,mean_nni,...,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1,label
0,train,01_tcp_ar,86,8643,s001,22/02/2012,t000,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,train,01_tcp_ar,86,8643,s001,22/02/2012,t000,1.0,10000.0,537.109375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,train,01_tcp_ar,86,8643,s001,22/02/2012,t000,2.0,20000.0,602.213542,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,train,01_tcp_ar,86,8643,s001,22/02/2012,t000,3.0,30000.0,564.453125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,train,01_tcp_ar,86,8643,s001,22/02/2012,t000,4.0,40000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [4]:
xqrs.describe(include='all')

,Set,Categorie Montage,Dossier Patient,Patient,Session,Date,File N°,interval_index,interval_start_time,mean_nni,...,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1,label
count,308945,308945,308945.000000,308945.000000,308945,308945,308945,308945.000000,3.089450e+05,271217.000000,...,190698.000000,190075.000000,232138.000000,2.325810e+05,232138.000000,2.655440e+05,232581.000000,232581.000000,232138.000000,304124.000000
unique,2,3,NaN,NaN,39,915,33,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,train,01_tcp_ar,NaN,NaN,s001,04/05/2014,t000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,251812,232247,NaN,NaN,59243,3516,84859,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,80.531823,8104.611789,NaN,NaN,NaN,46.626600,4.662660e+05,723.363894,...,357.658897,10.974066,5.106539,-inf,859.395808,inf,9.259266,39.410133,5.106539,0.067215
std,NaN,NaN,33.403979,3342.460990,NaN,NaN,NaN,46.158892,4.615889e+05,187.850494,...,845.394890,14.816918,3.031795,NaN,1230.536350,NaN,11.291463,43.771038,3.031795,0.244909
min,NaN,NaN,0.000000,2.000000,NaN,NaN,NaN,0.000000,0.000000e+00,0.000000,...,0.000000,0.037210,0.256758,-inf,0.361010,-0.000000e+00,0.000000,0.000000,0.256758,0.000000
25%,NaN,NaN,65.000000,6514.000000,NaN,NaN,NaN,15.000000,1.500000e+05,613.740809,...,19.189833,3.339780,3.110920,2.592831e+00,184.109288,3.778332e-01,2.070292,11.176479,3.110920,0.000000
50%,NaN,NaN,87.000000,8738.000000,NaN,NaN,NaN,33.000000,3.300000e+05,712.053571,...,88.703498,7.206585,4.333752,3.327306e+00,467.497737,6.154200e-01,5.256634,25.192242,4.333752,0.000000
75%,NaN,NaN,104.000000,10418.000000,NaN,NaN,NaN,60.000000,6.000000e+05,828.450521,...,338.092763,14.258062,6.191644,3.967830e+00,1049.816390,9.426080e-01,11.898158,51.315977,6.191644,0.000000


In [5]:
xqrs['Dossier Patient'] = xqrs['Dossier Patient'].astype(str)
xqrs['Patient'] = xqrs['Patient'].astype(str)

In [6]:
xqrs.describe(include='all')

,Set,Categorie Montage,Dossier Patient,Patient,Session,Date,File N°,interval_index,interval_start_time,mean_nni,...,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1,label
count,308945,308945,308945,308945,308945,308945,308945,308945.000000,3.089450e+05,271217.000000,...,190698.000000,190075.000000,232138.000000,2.325810e+05,232138.000000,2.655440e+05,232581.000000,232581.000000,232138.000000,304124.000000
unique,2,3,132,603,39,915,33,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,train,01_tcp_ar,104,9578,s001,04/05/2014,t000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,251812,232247,14065,11745,59243,3516,84859,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.626600,4.662660e+05,723.363894,...,357.658897,10.974066,5.106539,-inf,859.395808,inf,9.259266,39.410133,5.106539,0.067215
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.158892,4.615889e+05,187.850494,...,845.394890,14.816918,3.031795,NaN,1230.536350,NaN,11.291463,43.771038,3.031795,0.244909
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000e+00,0.000000,...,0.000000,0.037210,0.256758,-inf,0.361010,-0.000000e+00,0.000000,0.000000,0.256758,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.000000,1.500000e+05,613.740809,...,19.189833,3.339780,3.110920,2.592831e+00,184.109288,3.778332e-01,2.070292,11.176479,3.110920,0.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.000000,3.300000e+05,712.053571,...,88.703498,7.206585,4.333752,3.327306e+00,467.497737,6.154200e-01,5.256634,25.192242,4.333752,0.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.000000,6.000000e+05,828.450521,...,338.092763,14.258062,6.191644,3.967830e+00,1049.816390,9.426080e-01,11.898158,51.315977,6.191644,0.000000


**On cherche à savoir le pourcentage de crise d'un patient sur la durée totale du test**

In [7]:
percentage = xqrs[['Patient', 'label']].groupby(['Patient']).agg(['sum', 'count'])

In [8]:
percentage.columns

MultiIndex([('label',   'sum'),
            ('label', 'count')],
           )

In [9]:
percentage[('label', 'percent')] = (percentage[('label',   'sum')] / percentage[('label',   'count')]) * 100

In [10]:
percentage

label                 
               sum count    percent
Patient                            
10020    111.74689   351  31.836721
10022     55.13887  2740   2.012368
10023      0.00000   373   0.000000
10035      0.00000   132   0.000000
1006      17.65080   495   3.565818
...            ...   ...        ...
9906       0.00000   282   0.000000
9932       9.93164  1309   0.758720
9934      14.51991   683   2.125902
9958       0.00000   258   0.000000
9994      10.59720  1540   0.688130

[603 rows x 3 columns]

In [11]:
percentage.loc["2",:]

label  sum         59.059200
       count      298.000000
       percent     19.818523
Name: 2, dtype: float64

In [12]:
percentage

label                 
               sum count    percent
Patient                            
10020    111.74689   351  31.836721
10022     55.13887  2740   2.012368
10023      0.00000   373   0.000000
10035      0.00000   132   0.000000
1006      17.65080   495   3.565818
...            ...   ...        ...
9906       0.00000   282   0.000000
9932       9.93164  1309   0.758720
9934      14.51991   683   2.125902
9958       0.00000   258   0.000000
9994      10.59720  1540   0.688130

[603 rows x 3 columns]

**On cherche à savoir le pourcentage de NaN pour chaque features**

In [13]:
params = {}
for cols in xqrs.columns.tolist()[9:]:
    params[cols] = [lambda x: x.isnull().sum(), 'count']

In [14]:
count_nan = xqrs[['Patient'] + xqrs.columns.tolist()[9:]].groupby('Patient').agg(params)

In [15]:
for cols in xqrs.columns.tolist()[9:]:
    count_nan[(cols, 'count')] = (count_nan[(cols, '<lambda_0>')] + count_nan[(cols, 'count')])
    count_nan[(cols, 'percent')] = (count_nan[(cols, '<lambda_0>')] / count_nan[(cols, 'count')]) * 100

In [16]:
count_nan

mean_nni               sdnn               sdsd             nni_50  \
        <lambda_0>   count <lambda_0>   count <lambda_0>   count <lambda_0>   
Patient                                                                       
10020        154.0   354.0      154.0   354.0      154.0   354.0        0.0   
10022        432.0  2777.0      469.0  2777.0      469.0  2777.0       99.0   
10023          3.0   381.0        6.0   381.0        6.0   381.0        3.0   
10035          2.0   133.0        2.0   133.0        2.0   133.0        0.0   
1006          19.0   506.0       21.0   506.0       21.0   506.0        2.0   
...            ...     ...        ...     ...        ...     ...        ...   
9906          22.0   283.0       22.0   283.0       22.0   283.0        0.0   
9932         191.0  1327.0      199.0  1327.0      199.0  1327.0       30.0   
9934           4.0   693.0        7.0   693.0        7.0   693.0        4.0   
9958           2.0   259.0        2.0   259.0        2.0   259.0        0.0   
9994           3.0  1563.0       13.0  1563.0       13.0  1563.0        3.0   

                   pnni_50          ...        vlf lf_hf_ratio        csi  \
          count <lambda_0>   count  ...    percent     percent    percent   
Patient                             ...                                     
10020     354.0        0.0   354.0  ...  79.661017   79.661017  65.819209   
10022    2777.0       99.0  2777.0  ...  42.635938   42.671948  28.555996   
10023     381.0        3.0   381.0  ...  29.133858   29.133858  13.648294   
10035     133.0        0.0   133.0  ...  13.533835   13.533835   6.766917   
1006      506.0        2.0   506.0  ...  40.316206   40.316206  21.541502   
...         ...        ...     ...  ...        ...         ...        ...   
9906      283.0        0.0   283.0  ...  17.667845   17.667845  12.367491   
9932     1327.0       30.0  1327.0  ...  46.420497   46.420497  31.348907   
9934      693.0        4.0   693.0  ...  25.396825   25.396825  11.111111   
9958      259.0        0.0   259.0  ...   6.177606    6.177606   4.633205   
9994     1563.0        3.0  1563.0  ...  23.224568   23.224568  10.428663   

               cvi Modified_csi     sampen        sd1        sd2  \
           percent      percent    percent    percent    percent   
Patient                                                            
10020    65.819209    65.819209   5.932203  65.819209  65.819209   
10022    28.555996    28.555996  12.747569  28.555996  28.555996   
10023    13.648294    13.648294  13.648294  13.648294  13.648294   
10035     6.766917     6.766917   5.263158   6.766917   6.766917   
1006     21.541502    21.541502  17.786561  21.541502  21.541502   
...            ...          ...        ...        ...        ...   
9906     12.367491    12.367491   4.946996  12.367491  12.367491   
9932     31.198191    31.348907  14.920874  31.198191  31.198191   
9934     11.111111    11.111111  11.111111  11.111111  11.111111   
9958      4.633205     4.633205   2.702703   4.633205   4.633205   
9994     10.428663    10.428663  10.428663  10.428663  10.428663   

        ratio_sd2_sd1     label  
              percent   percent  
Patient                          
10020       65.819209  0.847458  
10022       28.555996  1.332373  
10023       13.648294  2.099738  
10035        6.766917  0.751880  
1006        21.541502  2.173913  
...               ...       ...  
9906        12.367491  0.353357  
9932        31.348907  1.356443  
9934        11.111111  1.443001  
9958         4.633205  0.386100  
9994        10.428663  1.471529  

[603 rows x 84 columns]

In [17]:
xqrs[xqrs['Patient'] == '11575']

,Set,Categorie Montage,Dossier Patient,Patient,Session,Date,File N°,interval_index,interval_start_time,mean_nni,...,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1,label
14073,train,01_tcp_ar,115,11575,s002,20/05/2014,t001,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
14074,train,01_tcp_ar,115,11575,s002,20/05/2014,t001,1.0,10000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
14075,train,01_tcp_ar,115,11575,s002,20/05/2014,t001,2.0,20000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
14076,train,01_tcp_ar,115,11575,s002,20/05/2014,t001,3.0,30000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
14077,train,01_tcp_ar,115,11575,s002,20/05/2014,t001,4.0,40000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
14078,train,01_tcp_ar,115,11575,s002,20/05/2014,t001,5.0,50000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
14079,train,01_tcp_ar,115,11575,s002,20/05/2014,t001,6.0,60000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
14080,train,01_tcp_ar,115,11575,s002,20/05/2014,t001,7.0,70000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,inf,NaN,NaN,NaN,0.0


**On crée un dictionnaire avec en clé le nom de colonne et en valeur l'ID des patients ayant plus de 40% de NaN dans cette colonne**

In [18]:
too_much_nan = {}
for cols in xqrs.columns.tolist()[9:]:
    too_much_nan[cols] = count_nan[count_nan[(cols, 'percent')] > 40].index

In [19]:
xqrs[xqrs['Patient'].isin(too_much_nan['mean_nni'].values.tolist())]

,Set,Categorie Montage,Dossier Patient,Patient,Session,Date,File N°,interval_index,interval_start_time,mean_nni,...,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1,label
2719,train,01_tcp_ar,120,12046,s001,09/10/2014,t000,0.0,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2720,train,01_tcp_ar,120,12046,s001,09/10/2014,t000,1.0,10000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2721,train,01_tcp_ar,120,12046,s001,09/10/2014,t000,2.0,20000.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2722,train,01_tcp_ar,120,12046,s001,09/10/2014,t000,3.0,30000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2723,train,01_tcp_ar,120,12046,s001,09/10/2014,t000,4.0,40000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308164,dev,02_tcp_le,19,1984,s001,17/02/2004,t001,141.0,1410000.0,728.615385,...,NaN,NaN,NaN,NaN,NaN,inf,NaN,NaN,NaN,0.0
308165,dev,02_tcp_le,19,1984,s001,17/02/2004,t001,142.0,1420000.0,727.692308,...,NaN,NaN,3.018914,4.091587,582.866953,0.869604,15.988507,48.267931,3.018914,0.0
308166,dev,02_tcp_le,19,1984,s001,17/02/2004,t001,143.0,1430000.0,NaN,...,NaN,NaN,3.138890,4.120422,638.815256,0.940555,16.209254,50.879070,3.138890,0.0
308167,dev,02_tcp_le,19,1984,s001,17/02/2004,t001,144.0,1440000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,inf,NaN,NaN,NaN,0.0


In [20]:
len(too_much_nan['mean_nni'].values.tolist())

45

**On crée un dictionnaire avec en clé le nom de colonne et en valeur l'ID des patients ayant moins de 10% de NaN dans cette colonne**

In [21]:
less_10_nan = {}
for cols in xqrs.columns.tolist()[9:]:
    less_10_nan[cols] = set(count_nan[count_nan[(cols, 'percent')] < 10].index.tolist())

In [22]:
len(less_10_nan['mean_nni'])

419

**On cherche les patients apparaissant dans chaque colonne**

In [23]:
set.intersection(*less_10_nan.values())

{'10301',
 '10547',
 '11063',
 '11149',
 '11596',
 '12070',
 '1482',
 '1690',
 '2126',
 '2225',
 '3635',
 '5034',
 '5411',
 '5469',
 '5943',
 '6607',
 '7937',
 '8188',
 '850',
 '8550',
 '8573',
 '9762',
 '9958'}

In [24]:
xqrs[xqrs['Patient'] == '5943']

,Set,Categorie Montage,Dossier Patient,Patient,Session,Date,File N°,interval_index,interval_start_time,mean_nni,...,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1,label
305207,dev,02_tcp_le,59,5943,s005,08/07/2009,t000,0.0,0.0,543.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
305208,dev,02_tcp_le,59,5943,s005,08/07/2009,t000,1.0,10000.0,543.555556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
305209,dev,02_tcp_le,59,5943,s005,08/07/2009,t000,2.0,20000.0,533.263158,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
305210,dev,02_tcp_le,59,5943,s005,08/07/2009,t000,3.0,30000.0,545.555556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
305211,dev,02_tcp_le,59,5943,s005,08/07/2009,t000,4.0,40000.0,545.777778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305607,dev,02_tcp_le,59,5943,s001,28/06/2009,t000,275.0,2750000.0,642.250000,...,2.577752,4.436143,2.442496,2.283853,52.927012,0.371808,2.217939,5.417308,2.442496,0.0
305608,dev,02_tcp_le,59,5943,s001,28/06/2009,t000,276.0,2760000.0,645.866667,...,1.837404,3.732497,1.986720,2.231558,36.558122,0.458811,2.315531,4.600311,1.986720,0.0
305609,dev,02_tcp_le,59,5943,s001,28/06/2009,t000,277.0,2770000.0,642.250000,...,1.685554,6.365177,1.727814,2.178057,27.878803,0.572735,2.334641,4.033826,1.727814,0.0
305610,dev,02_tcp_le,59,5943,s001,28/06/2009,t000,278.0,2780000.0,642.666667,...,0.703117,3.571622,1.423337,2.204566,21.490413,0.803932,2.651974,3.774653,1.423337,0.0


In [25]:
xqrs[(xqrs['Patient'] == '5943') & (xqrs['Session'] == 's001')]['label'].sum()

161.08875999999998

In [26]:
xqrs[(xqrs['Patient'] == '5943') & (xqrs['Session'] == 's001')]['label'].value_counts()

1.00000    150
0.00000    109
0.03073      1
0.85749      1
0.64440      1
0.57801      1
0.27213      1
0.75530      1
0.91171      1
0.90000      1
0.53322      1
0.67949      1
0.35418      1
0.56343      1
0.70401      1
0.73061      1
0.58885      1
0.36318      1
0.54705      1
0.36499      1
0.40400      1
0.07320      1
0.23278      1
Name: label, dtype: int64

In [27]:
XQRS_FEATS_FILE = "../feats-v0_4/dev/02_tcp_le/059/00005943/s001_2009_06_28/00005943_s001_t000-xqrs.json"
PAN_TOMPKINS_FEATS_FILE = "../feats-v0_4/dev/02_tcp_le/059/00005943/s001_2009_06_28/00005943_s001_t000-pan.json"
SWT_FILE = "../feats-v0_4/dev/02_tcp_le/059/00005943/s001_2009_06_28/00005943_s001_t000-swt.json"

xqrs_data = json.load(open(XQRS_FEATS_FILE, "r"))
panTompkins_data = json.load(open(PAN_TOMPKINS_FEATS_FILE, "r"))
swt_data = json.load(open(SWT_FILE, "r"))

xqrs_features = np.array(xqrs_data["features"])
panTompkins_features = np.array(panTompkins_data["features"])
swt_features = np.array(swt_data["features"])

xqrs_features_tp = np.transpose(xqrs_features)
panTompkins_features_tp = np.transpose(panTompkins_features)
swt_features_tp =np.transpose(swt_features)

for i in range(0, len(xqrs_data["keys"][:-1])):
    plt.figure(1, figsize = (20,2))
    #print(xqrs_data["keys"][i])
    plt.title(xqrs_data["keys"][i])
    plt.plot(xqrs_features_tp[i], color='red', alpha=0.5, label="xqrs")
    plt.plot(panTompkins_features_tp[i], color='blue', alpha=0.8, label="pan-tompkins")
    plt.plot(swt_features_tp[i], color='green', alpha=0.5, label="swt")
    plt.legend(bbox_to_anchor=(0.80, 1), loc='upper left', fontsize='x-large')
    
    for j in range(0, len(xqrs_features_tp[29])):
        plt.axvspan(j, (j + 1), facecolor='grey', alpha=0.5 * xqrs_features_tp[29][j])
    plt.plot()
    plt.show()

NameError: name 'json' is not defined

**On cherche à identifier les meilleurs enregistrements**

In [28]:
xqrs['Key'] = xqrs.Set + '_' + xqrs['Categorie Montage'] + '_' + xqrs['Patient'] + '_' + xqrs.Session + '_' + xqrs['File N°']

In [29]:
cols = xqrs.columns.tolist()
cols = cols[-1:] + cols[:-1]
xqrs = xqrs[cols]
xqrs.head()

,Key,Set,Categorie Montage,Dossier Patient,Patient,Session,Date,File N°,interval_index,interval_start_time,...,vlf,lf_hf_ratio,csi,cvi,Modified_csi,sampen,sd1,sd2,ratio_sd2_sd1,label
0,train_01_tcp_ar_8643_s001_t000,train,01_tcp_ar,86,8643,s001,22/02/2012,t000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,train_01_tcp_ar_8643_s001_t000,train,01_tcp_ar,86,8643,s001,22/02/2012,t000,1.0,10000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,train_01_tcp_ar_8643_s001_t000,train,01_tcp_ar,86,8643,s001,22/02/2012,t000,2.0,20000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,train_01_tcp_ar_8643_s001_t000,train,01_tcp_ar,86,8643,s001,22/02/2012,t000,3.0,30000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,train_01_tcp_ar_8643_s001_t000,train,01_tcp_ar,86,8643,s001,22/02/2012,t000,4.0,40000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [30]:
params = {}
for cols in xqrs.columns.tolist()[10:]:
    params[cols] = [lambda x: x.isnull().sum(), 
                    lambda x: x.count() + x.isnull().sum(), 
                    lambda x: (x.isnull().sum() / (x.count() + x.isnull().sum())) * 100]

In [31]:
count_nan_by_key = xqrs[['Key'] + xqrs.columns.tolist()[10:]]\
                                              .groupby('Key')\
                                              .agg(params)\
                                              .rename(columns={'<lambda_0>': 'NaN',
                                                               '<lambda_1>': 'total',
                                                               '<lambda_2>': '%'})

In [32]:
count_nan_by_key

mean_nni                    sdnn         \
                                      NaN  total          %   NaN  total   
Key                                                                        
dev_01_tcp_ar_10022_s001_t000         0.0    3.0   0.000000   0.0    3.0   
dev_01_tcp_ar_10022_s001_t001         7.0  127.0   5.511811  10.0  127.0   
dev_01_tcp_ar_10022_s002_t000         2.0   61.0   3.278689   2.0   61.0   
dev_01_tcp_ar_10022_s002_t001         1.0   61.0   1.639344   2.0   61.0   
dev_01_tcp_ar_10022_s002_t002         0.0   61.0   0.000000   1.0   61.0   
...                                   ...    ...        ...   ...    ...   
train_03_tcp_ar_a_9162_s003_t006     46.0   61.0  75.409836  47.0   61.0   
train_03_tcp_ar_a_9162_s003_t007     12.0   17.0  70.588235  12.0   17.0   
train_03_tcp_ar_a_9162_s003_t008     15.0   61.0  24.590164  16.0   61.0   
train_03_tcp_ar_a_9162_s003_t009     28.0   30.0  93.333333  29.0   30.0   
train_03_tcp_ar_a_9162_s003_t010     13.0   61.0  21.311475  14.0   61.0   

                                             sdsd                   nni_50  \
                                          %   NaN  total          %    NaN   
Key                                                                          
dev_01_tcp_ar_10022_s001_t000      0.000000   0.0    3.0   0.000000    0.0   
dev_01_tcp_ar_10022_s001_t001      7.874016  10.0  127.0   7.874016    3.0   
dev_01_tcp_ar_10022_s002_t000      3.278689   2.0   61.0   3.278689    1.0   
dev_01_tcp_ar_10022_s002_t001      3.278689   2.0   61.0   3.278689    0.0   
dev_01_tcp_ar_10022_s002_t002      1.639344   1.0   61.0   1.639344    0.0   
...                                     ...   ...    ...        ...    ...   
train_03_tcp_ar_a_9162_s003_t006  77.049180  47.0   61.0  77.049180    0.0   
train_03_tcp_ar_a_9162_s003_t007  70.588235  12.0   17.0  70.588235    0.0   
train_03_tcp_ar_a_9162_s003_t008  26.229508  16.0   61.0  26.229508    0.0   
train_03_tcp_ar_a_9162_s003_t009  96.666667  29.0   30.0  96.666667   27.0   
train_03_tcp_ar_a_9162_s003_t010  22.950820  14.0   61.0  22.950820    0.0   

                                  ...         sd1   sd2                     \
                                  ...           %   NaN  total           %   
Key                               ...                                        
dev_01_tcp_ar_10022_s001_t000     ...  100.000000   3.0    3.0  100.000000   
dev_01_tcp_ar_10022_s001_t001     ...   12.598425  16.0  127.0   12.598425   
dev_01_tcp_ar_10022_s002_t000     ...   13.114754   8.0   61.0   13.114754   
dev_01_tcp_ar_10022_s002_t001     ...   11.475410   7.0   61.0   11.475410   
dev_01_tcp_ar_10022_s002_t002     ...   11.475410   7.0   61.0   11.475410   
...                               ...         ...   ...    ...         ...   
train_03_tcp_ar_a_9162_s003_t006  ...   98.360656  60.0   61.0   98.360656   
train_03_tcp_ar_a_9162_s003_t007  ...  100.000000  17.0   17.0  100.000000   
train_03_tcp_ar_a_9162_s003_t008  ...   45.901639  28.0   61.0   45.901639   
train_03_tcp_ar_a_9162_s003_t009  ...  100.000000  30.0   30.0  100.000000   
train_03_tcp_ar_a_9162_s003_t010  ...   50.819672  31.0   61.0   50.819672   

                                 ratio_sd2_sd1                    label  \
                                           NaN  total           %   NaN   
Key                                                                       
dev_01_tcp_ar_10022_s001_t000              3.0    3.0  100.000000   0.0   
dev_01_tcp_ar_10022_s001_t001             16.0  127.0   12.598425   0.0   
dev_01_tcp_ar_10022_s002_t000              8.0   61.0   13.114754   1.0   
dev_01_tcp_ar_10022_s002_t001              7.0   61.0   11.475410   1.0   
dev_01_tcp_ar_10022_s002_t002              7.0   61.0   11.475410   1.0   
...                                        ...    ...         ...   ...   
train_03_tcp_ar_a_9162_s003_t006          60.0   61.0   98.360656   1.0   
train_03_tcp_ar_a_9162_s003_t007

In [33]:
total = np.zeros(5080,)
i = 0
for cols in xqrs.columns.tolist()[10:]:
    sub_total = []
    for index, row in count_nan_by_key[(cols, '%')].iteritems():
        sub_total.append(row)
    i += 1
    total = np.add(total, sub_total)


total = total / 28

In [34]:
count_nan_by_key[('Total', 'Percent')] = total

In [35]:
count_nan_by_key

mean_nni                    sdnn         \
                                      NaN  total          %   NaN  total   
Key                                                                        
dev_01_tcp_ar_10022_s001_t000         0.0    3.0   0.000000   0.0    3.0   
dev_01_tcp_ar_10022_s001_t001         7.0  127.0   5.511811  10.0  127.0   
dev_01_tcp_ar_10022_s002_t000         2.0   61.0   3.278689   2.0   61.0   
dev_01_tcp_ar_10022_s002_t001         1.0   61.0   1.639344   2.0   61.0   
dev_01_tcp_ar_10022_s002_t002         0.0   61.0   0.000000   1.0   61.0   
...                                   ...    ...        ...   ...    ...   
train_03_tcp_ar_a_9162_s003_t006     46.0   61.0  75.409836  47.0   61.0   
train_03_tcp_ar_a_9162_s003_t007     12.0   17.0  70.588235  12.0   17.0   
train_03_tcp_ar_a_9162_s003_t008     15.0   61.0  24.590164  16.0   61.0   
train_03_tcp_ar_a_9162_s003_t009     28.0   30.0  93.333333  29.0   30.0   
train_03_tcp_ar_a_9162_s003_t010     13.0   61.0  21.311475  14.0   61.0   

                                             sdsd                   nni_50  \
                                          %   NaN  total          %    NaN   
Key                                                                          
dev_01_tcp_ar_10022_s001_t000      0.000000   0.0    3.0   0.000000    0.0   
dev_01_tcp_ar_10022_s001_t001      7.874016  10.0  127.0   7.874016    3.0   
dev_01_tcp_ar_10022_s002_t000      3.278689   2.0   61.0   3.278689    1.0   
dev_01_tcp_ar_10022_s002_t001      3.278689   2.0   61.0   3.278689    0.0   
dev_01_tcp_ar_10022_s002_t002      1.639344   1.0   61.0   1.639344    0.0   
...                                     ...   ...    ...        ...    ...   
train_03_tcp_ar_a_9162_s003_t006  77.049180  47.0   61.0  77.049180    0.0   
train_03_tcp_ar_a_9162_s003_t007  70.588235  12.0   17.0  70.588235    0.0   
train_03_tcp_ar_a_9162_s003_t008  26.229508  16.0   61.0  26.229508    0.0   
train_03_tcp_ar_a_9162_s003_t009  96.666667  29.0   30.0  96.666667   27.0   
train_03_tcp_ar_a_9162_s003_t010  22.950820  14.0   61.0  22.950820    0.0   

                                  ...   sd2                    ratio_sd2_sd1  \
                                  ...   NaN  total           %           NaN   
Key                               ...                                          
dev_01_tcp_ar_10022_s001_t000     ...   3.0    3.0  100.000000           3.0   
dev_01_tcp_ar_10022_s001_t001     ...  16.0  127.0   12.598425          16.0   
dev_01_tcp_ar_10022_s002_t000     ...   8.0   61.0   13.114754           8.0   
dev_01_tcp_ar_10022_s002_t001     ...   7.0   61.0   11.475410           7.0   
dev_01_tcp_ar_10022_s002_t002     ...   7.0   61.0   11.475410           7.0   
...                               ...   ...    ...         ...           ...   
train_03_tcp_ar_a_9162_s003_t006  ...  60.0   61.0   98.360656          60.0   
train_03_tcp_ar_a_9162_s003_t007  ...  17.0   17.0  100.000000          17.0   
train_03_tcp_ar_a_9162_s003_t008  ...  28.0   61.0   45.901639          28.0   
train_03_tcp_ar_a_9162_s003_t009  ...  30.0   30.0  100.000000          30.0   
train_03_tcp_ar_a_9162_s003_t010  ...  31.0   61.0   50.819672          31.0   

                                                    label                   \
                                  total           %   NaN  total         %   
Key                                                                          
dev_01_tcp_ar_10022_s001_t000       3.0  100.000000   0.0    3.0  0.000000   
dev_01_tcp_ar_10022_s001_t001     127.0   12.598425   0.0  127.0  0.000000   
dev_01_tcp_ar_10022_s002_t000      61.0   13.114754   1.0   61.0  1.639344   
dev_01_tcp_ar_10022_s002_t001      61.0   11.475410   1.0   61.0  1.639344   
dev_01_tcp_ar_10022_s002_t002      61.0   11.475410   1.0   61.0  1.639344   
...                                 ...         ...   ...    ...       ...   
train_03_tcp_ar_a_9162_s003_t006   61.0   98.360656 

In [46]:
count_nan_by_key[count_nan_by_key[('Total', 'Percent')] < 2]

mean_nni                  sdnn         \
                                       NaN  total         %  NaN  total   
Key                                                                       
dev_01_tcp_ar_8544_s002_t005           0.0  312.0  0.000000  0.0  312.0   
dev_01_tcp_ar_9578_s011_t008           0.0  252.0  0.000000  0.0  252.0   
dev_01_tcp_ar_9578_s017_t004           0.0  206.0  0.000000  0.0  206.0   
dev_01_tcp_ar_9578_s021_t001           0.0  341.0  0.000000  0.0  341.0   
dev_01_tcp_ar_9842_s002_t001           0.0  341.0  0.000000  0.0  341.0   
dev_02_tcp_le_1770_s003_t000           0.0  237.0  0.000000  0.0  237.0   
dev_02_tcp_le_5943_s001_t000           1.0  280.0  0.357143  1.0  280.0   
train_01_tcp_ar_10209_s008_t000        0.0  303.0  0.000000  0.0  303.0   
train_01_tcp_ar_10489_s002_t001        0.0  232.0  0.000000  0.0  232.0   
train_01_tcp_ar_10969_s008_t001        0.0  239.0  0.000000  0.0  239.0   
train_01_tcp_ar_11063_s002_t001        0.0  237.0  0.000000  0.0  237.0   
train_01_tcp_ar_11596_s002_t001        1.0  357.0  0.280112  1.0  357.0   
train_01_tcp_ar_11869_s001_t004        0.0  211.0  0.000000  0.0  211.0   
train_01_tcp_ar_11870_s005_t002        1.0  235.0  0.425532  1.0  235.0   
train_01_tcp_ar_1482_s004_t001         3.0  346.0  0.867052  3.0  346.0   
train_01_tcp_ar_7032_s008_t000         0.0  245.0  0.000000  0.0  245.0   
train_01_tcp_ar_8164_s004_t003         0.0  225.0  0.000000  0.0  225.0   
train_01_tcp_ar_8476_s007_t001         0.0  212.0  0.000000  0.0  212.0   
train_01_tcp_ar_8480_s002_t004         0.0  281.0  0.000000  0.0  281.0   
train_01_tcp_ar_8480_s004_t001         0.0  268.0  0.000000  0.0  268.0   
train_01_tcp_ar_8552_s007_t004         1.0  260.0  0.384615  2.0  260.0   
train_02_tcp_le_1690_s001_t000         0.0  351.0  0.000000  0.0  351.0   
train_02_tcp_le_2126_s001_t000         0.0  254.0  0.000000  0.0  254.0   
train_02_tcp_le_499_s002_t002          0.0  235.0  0.000000  0.0  235.0   
train_02_tcp_le_5411_s002_t000         0.0  244.0  0.000000  0.0  244.0   
train_03_tcp_ar_a_13112_s002_t001      2.0  272.0  0.735294  2.0  272.0   

                                            sdsd                  nni_50  ...  \
                                          %  NaN  total         %    NaN  ...   
Key                                                                       ...   
dev_01_tcp_ar_8544_s002_t005       0.000000  0.0  312.0  0.000000    0.0  ...   
dev_01_tcp_ar_9578_s011_t008       0.000000  0.0  252.0  0.000000    0.0  ...   
dev_01_tcp_ar_9578_s017_t004       0.000000  0.0  206.0  0.000000    0.0  ...   
dev_01_tcp_ar_9578_s021_t001       0.000000  0.0  341.0  0.000000    0.0  ...   
dev_01_tcp_ar_9842_s002_t001       0.000000  0.0  341.0  0.000000    0.0  ...   
dev_02_tcp_le_1770_s003_t000       0.000000  0.0  237.0  0.000000    0.0  ...   
dev_02_tcp_le_5943_s001_t000       0.357143  1.0  280.0  0.357143    0.0  ...   
train_01_tcp_ar_10209_s008_t000    0.000000  0.0  303.0  0.000000    0.0  ...   
train_01_tcp_ar_10489_s002_t001    0.000000  0.0  232.0  0.000000    0.0  ...   
train_01_tcp_ar_10969_s008_t001    0.000000  0.0  239.0  0.000000    0.0  ...   
train_01_tcp_ar_11063_s002_t001    0.000000  0.0  237.0  0.000000    0.0  ...   
train_01_tcp_ar_11596_s002_t001    0.280112  1.0  357.0  0.280112    0.0  ...   
train_01_tcp_ar_11869_s001_t004    0.000000  0.0  211.0  0.000000    0.0  ...   
train_01_tcp_ar_11870_s005_t002    0.425532  1.0  235.0  0.425532    1.0  ...   
train_01_tcp_ar_1482_s004_t001     0.867052  3.0  346.0  0.867052    0.0  ...   
train_01_tcp_ar_7032_s008_t000     0.000000  0.0  245.0  0.000000    0.0  ...   
train_01_tcp_ar_8164_s004_t003     0.000000  0.0  225.0  0.000000    0.0  ...   
train_01_tcp_ar_8476_s007_t001     0.000000  0.0  212.0  0.000000    0.0  ...   
train_01_tcp_ar_8480_s002_t004     0.000000  0.0  281.0  0.000000    0.0  ...   
train_01_tcp_ar_8480_s004_t001     0.000000  0.0  268.0  0.000000    0.0  ..

In [47]:
count_nan_by_key[count_nan_by_key[('Total', 'Percent')] < 2].index

Index(['dev_01_tcp_ar_8544_s002_t005', 'dev_01_tcp_ar_9578_s011_t008',
       'dev_01_tcp_ar_9578_s017_t004', 'dev_01_tcp_ar_9578_s021_t001',
       'dev_01_tcp_ar_9842_s002_t001', 'dev_02_tcp_le_1770_s003_t000',
       'dev_02_tcp_le_5943_s001_t000', 'train_01_tcp_ar_10209_s008_t000',
       'train_01_tcp_ar_10489_s002_t001', 'train_01_tcp_ar_10969_s008_t001',
       'train_01_tcp_ar_11063_s002_t001', 'train_01_tcp_ar_11596_s002_t001',
       'train_01_tcp_ar_11869_s001_t004', 'train_01_tcp_ar_11870_s005_t002',
       'train_01_tcp_ar_1482_s004_t001', 'train_01_tcp_ar_7032_s008_t000',
       'train_01_tcp_ar_8164_s004_t003', 'train_01_tcp_ar_8476_s007_t001',
       'train_01_tcp_ar_8480_s002_t004', 'train_01_tcp_ar_8480_s004_t001',
       'train_01_tcp_ar_8552_s007_t004', 'train_02_tcp_le_1690_s001_t000',
       'train_02_tcp_le_2126_s001_t000', 'train_02_tcp_le_499_s002_t002',
       'train_02_tcp_le_5411_s002_t000', 'train_03_tcp_ar_a_13112_s002_t001'],
      dtype='object', name='K

In [37]:
from functions import get_good_file

ModuleNotFoundError: No module named 'tqdm'

In [38]:
paths, cle_exams, coefs = get_good_file('../res-v0_4', 0.8)

NameError: name 'get_good_file' is not defined

In [39]:
keys[:5]

NameError: name 'keys' is not defined

In [40]:
paths[0]

NameError: name 'paths' is not defined

In [41]:
count_nan_by_key.shape

(5080, 85)

In [42]:
filter_count_nan_by_key = count_nan_by_key[count_nan_by_key.index.isin(keys)]

NameError: name 'keys' is not defined

In [43]:
filter_count_nan_by_key.shape

NameError: name 'filter_count_nan_by_key' is not defined